In [1]:
!pip install pymongo
from pymongo import MongoClient

In [2]:
host="rhea.isegi.unl.pt"
port="28011"
user="GROUP_11"
password="MTM0MzQ5MzI3NjY4ODM1MzQ2ODc2OTMzMjgyNjQ3NjYyNDcyNjUw"
protocol="mongodb"
client = MongoClient(f"{protocol}://{user}:{password}@{host}:{port}")

In [3]:
db = client.contracts
eu = db.eu

In [4]:
bot_year = 2008
top_year = 2020
country_list = ['NO', 'HR', 'HU', 'CH', 'CZ', 'RO', 'LV', 'GR', 'UK', 'SI', 'LT', 'ES', 'FR', 'IE', 'SE', 'NL',
                'PT', 'PL', 'DK', 'MK', 'DE', 'IT', 'BG', 'CY', 'AT', 'LU', 'BE', 'FI', 'EE', 'SK', 'MT', 'LI',
                'IS']

11. The count of contracts per country ('ISO_COUNTRY_CODE')

In [6]:
match = {
    '$match': {
        '$and': [{'YEAR': {'$gte': bot_year}}, {'YEAR': {'$lte': top_year}}],
        'ISO_COUNTRY_CODE': {'$in': country_list}
    }
}

group_country_count = {
    '$group':{
        '_id':'$ISO_COUNTRY_CODE',
        'COUNT':{'$sum':1}
    }
}

project = {
    '$project':{
        'country':'$_id',
        'count':'$COUNT',
        '_id':False
    }
}
pipeline = [match, group_country_count, project]

list_documents = list(eu.aggregate(pipeline))

list_documents

[{'country': 'IS', 'count': 1950},
 {'country': 'EE', 'count': 26992},
 {'country': 'BE', 'count': 77642},
 {'country': 'HU', 'count': 72794},
 {'country': 'SK', 'count': 40809},
 {'country': 'NO', 'count': 39049},
 {'country': 'IT', 'count': 204976},
 {'country': 'UK', 'count': 386501},
 {'country': 'IE', 'count': 32686},
 {'country': 'PT', 'count': 35912},
 {'country': 'LV', 'count': 123398},
 {'country': 'CZ', 'count': 128369},
 {'country': 'MT', 'count': 4548},
 {'country': 'LU', 'count': 9961},
 {'country': 'RO', 'count': 297141},
 {'country': 'HR', 'count': 40462},
 {'country': 'LI', 'count': 346},
 {'country': 'SE', 'count': 112423},
 {'country': 'ES', 'count': 251927},
 {'country': 'AT', 'count': 37616},
 {'country': 'FI', 'count': 70955},
 {'country': 'DK', 'count': 64496},
 {'country': 'LT', 'count': 117109},
 {'country': 'PL', 'count': 1238186},
 {'country': 'SI', 'count': 150639},
 {'country': 'MK', 'count': 9678},
 {'country': 'CY', 'count': 8865},
 {'country': 'NL', 'coun

12. Returns the average 'Value_Euro' for each country, return the highest 5 countries

In [10]:
match = {
    '$match': {
        '$and': [{'YEAR': {'$gte': bot_year}}, {'YEAR': {'$lte': top_year}}],
        'ISO_COUNTRY_CODE': {'$in': country_list}
    }
}

project = {
    '$project':{
        '_id':False,
        'VALUE_EURO':True,
        'ISO_COUNTRY_CODE':True
    }
}

group = {
    '$group':{
        '_id':'$ISO_COUNTRY_CODE',
        'AVERAGE':{'$avg':'$VALUE_EURO'}
    }
}
sort = {
    '$sort':{
        'AVERAGE': -1
    }
}

limit = {
    '$limit':5
}

project_2 = {
    '$project':{
        '_id':False,
        'country':'$_id',
        'avg':'$AVERAGE'
    }
}

pipeline = [match, project, group, sort, limit, project_2]

list_documents = list(eu.aggregate(pipeline))

list_documents

[{'country': 'HU', 'avg': 2.3457793956061657e+22},
 {'country': 'DE', 'avg': 1.565263668670854e+20},
 {'country': 'BE', 'avg': 2.796330886808232e+18},
 {'country': 'LU', 'avg': 4798276236001678.0},
 {'country': 'GR', 'avg': 3433382284308.2563}]

13. Returns the average 'VALUE_EURO' for each country, return the lowest 5 countries
 

In [11]:
match = {
    '$match': {
        '$and': [{'YEAR': {'$gte': bot_year}}, {'YEAR': {'$lte': top_year}}],
        'ISO_COUNTRY_CODE': {'$in': country_list}
    }
}

project = {
    '$project':{
        '_id':False,
        'VALUE_EURO':True,
        'ISO_COUNTRY_CODE':True
    }
}

group = {
    '$group':{
        '_id':'$ISO_COUNTRY_CODE',
        'AVERAGE':{'$avg':'$VALUE_EURO'}
    }
}
sort = {
    '$sort':{
        'AVERAGE': 1
    }
}

limit = {
    '$limit':5
}

project_2 = {
    '$project':{
        '_id':False,
        'country':'$_id',
        'avg':'$AVERAGE'
    }
}
pipeline = [match, project, group, sort, limit, project_2]

list_documents = list(eu.aggregate(pipeline))

list_documents

[{'country': 'MK', 'avg': 876557.29},
 {'country': 'BG', 'avg': 1209211.7481247569},
 {'country': 'SI', 'avg': 1212932.4157451054},
 {'country': 'CY', 'avg': 1363493.8052469224},
 {'country': 'MT', 'avg': 1533678.4470194138}]

14. For each country get the sum of the respective contracts 'VALUE_EURO' which recieved european funds ('B_EU_FUNDS') 

In [12]:
match = {
    '$match': {
        '$and': [{'YEAR': {'$gte': bot_year}}, {'YEAR': {'$lte': top_year}}],
        'ISO_COUNTRY_CODE': {'$in': country_list},
        'B_EU_FUNDS': 'Y'
    }
}

project = {
    '$project':{
        '_id':False,
        'VALUE_EURO':True,
        'ISO_COUNTRY_CODE':True
    }
}

group = {
    '$group':{
        '_id':'$ISO_COUNTRY_CODE',
        'SUM':{'$sum':'$VALUE_EURO'}
    }
}


project_2 = {
    '$project':{
        '_id':False,
        'country':'$_id',
        'sum':'$SUM'
    }
}
pipeline = [match, project, group, project_2]

list_documents = list(eu.aggregate(pipeline))

list_documents

[{'country': 'LT', 'sum': 9486496427.27},
 {'country': 'MK', 'sum': 452344940.36},
 {'country': 'SI', 'sum': 5441697306.31},
 {'country': 'NL', 'sum': 2704277735.66},
 {'country': 'CY', 'sum': 627546920.1},
 {'country': 'RO', 'sum': 29452802221.11},
 {'country': 'HR', 'sum': 1157493694.04},
 {'country': 'AT', 'sum': 875062978.46},
 {'country': 'ES', 'sum': 70976159414.11},
 {'country': 'SE', 'sum': 894378979.03},
 {'country': 'PL', 'sum': 133664049320.58},
 {'country': 'DK', 'sum': 6621970949.15},
 {'country': 'FI', 'sum': 2490943993.64},
 {'country': 'GR', 'sum': 50167185957.3},
 {'country': 'DE', 'sum': 13229821653.08},
 {'country': 'FR', 'sum': 109474293715.69},
 {'country': 'BG', 'sum': 10996693088.45},
 {'country': 'CH', 'sum': 0},
 {'country': 'BE', 'sum': 8.181162968574296e+17},
 {'country': 'HU', 'sum': 1.3471576490942093e+27},
 {'country': 'SK', 'sum': 33734105558.82},
 {'country': 'NO', 'sum': 27142128.09},
 {'country': 'IT', 'sum': 72731128631.57},
 {'country': 'EE', 'sum': 

15. 5 descriptive metrics of the contracts related to the Company, the average of:
    1. Each Company's contracts average spending ('VALUE_EURO') 
    1. Each Company's contract count 
    1. Each Company's contracts average NUMBER_OFFERS' 
    1. Each Company's contracts average VALUE_EURO' with 'B_EU_FUNDS' 
    1. Each Company's contracts average 'VALUE_EURO' without 'B_EU_FUNDS'

In [ ]:
match = {
    '$match': {
        '$and': [{'YEAR': {'$gte': bot_year}}, {'YEAR': {'$lte': top_year}}],
        'ISO_COUNTRY_CODE': {'$in': country_list},
    }
}
project = {
    '$project':{
        '_id':False,
        'VALUE_EURO':True,
        'ISO_COUNTRY_CODE':True,
        'CAE_NAME':True
    }
}

group = {
    '$group':{
        '_id':'$ISO_COUNTRY_CODE',
        'SUM':{'$sum':'$VALUE_EURO'}
    }
}


project_2 = {
    '$project':{
        '_id':False,
        'country':'$_id',
        'sum':'$SUM'
    }
}
pipeline = [match, project, group, project_2]

list_documents = list(eu.aggregate(pipeline))

list_documents

16. Returns the average 'VALUE_EURO' for company ('CAE_NAME') return the highest 5 companies

In [15]:
match = {
    '$match': {
        '$and': [{'YEAR': {'$gte': bot_year}}, {'YEAR': {'$lte': top_year}}],
        'ISO_COUNTRY_CODE': {'$in': country_list}
    }
}
project = {
    '$project':{
        '_id':False,
        'VALUE_EURO':True,
        'CAE_NAME':True
    }
}

group = {
    '$group':{
        '_id':'$CAE_NAME',
        'AVERAGE':{'$avg':'$VALUE_EURO'}
    }
}
sort = {
    '$sort':{
        'AVERAGE': -1
    }
}

limit = {
    '$limit':5
}


project_2 = {
    '$project':{
        '_id':False,
        'company':'$_id',
        'avg':'$AVERAGE'
    }
}

pipeline = [match, project, group, sort, limit, project_2]

list_documents = list(eu.aggregate(pipeline))

list_documents

[{'company': 'Közép-dunántúli Vízügyi Igazgatóság',
  'avg': 2.2452627484874678e+26},
 {'company': 'European Aviation Safety Agency', 'avg': 3e+23},
 {'company': 'European Commission, European Administrative School',
  'avg': 4.3435e+21},
 {'company': 'European Commission, Joint Research Centre, Institute for Reference Materials and Measurements',
  'avg': 5.8582930565313934e+20},
 {'company': 'European Commission, Directorate-General for Employment, Social Affairs and Inclusion, Unit G3',
  'avg': 4.66764814e+17}]

17. Returns the average 'VALUE_EURO' for company ('CAE_NAME') return the lowest 5 companies

In [16]:
match = {
    '$match': {
        '$and': [{'YEAR': {'$gte': bot_year}}, {'YEAR': {'$lte': top_year}}],
        'ISO_COUNTRY_CODE': {'$in': country_list}
    }
}
project = {
    '$project':{
        '_id':False,
        'VALUE_EURO':True,
        'CAE_NAME':True
    }
}
group = {
    '$group':{
        '_id':'$CAE_NAME',
        'AVERAGE':{'$avg':'$VALUE_EURO'}
    }
}
sort = {
    '$sort':{
        'AVERAGE': 1
    }
}

limit = {
    '$limit':5
}


project_2 = {
    '$project':{
        '_id':False,
        'company':'$_id',
        'avg':'$AVERAGE'
    }
}

pipeline = [match, project, group, sort, limit, project_2]

list_documents = list(eu.aggregate(pipeline))

list_documents

[{'company': 'Stadt Dülmen---Weiterführung der Planung und Baubeaufsichtigung eines Straßen- und Brückenbauprojektes',
  'avg': None},
 {'company': 'Lausanne Région', 'avg': None},
 {'company': 'EMA/dcdirisi/dirisi Brest', 'avg': None},
 {'company': 'Secretary of State for Health acting through the NHS Commercial Medicines Unit',
  'avg': None},
 {'company': 'KRH-Servicegesellschaft GmbH', 'avg': None}]

18. Returns the count of contracts for each company 'CAE_NAME', for the 15 companies with the most contracts

In [17]:
match = {
    '$match': {
        '$and': [{'YEAR': {'$gte': bot_year}}, {'YEAR': {'$lte': top_year}}],
        'ISO_COUNTRY_CODE': {'$in': country_list}
    }
}
project = {
    '$project':{
        '_id':False,
        'CAE_NAME':True
    }
}
group = {
    '$group':{
        '_id':'$CAE_NAME',
        'COUNT':{'$sum':1}
    }
}
sort = {
    '$sort':{
        'COUNT': -1
    }
}

limit = {
    '$limit':15
}


project_2 = {
    '$project':{
        '_id':False,
        'company':'$_id',
        'count':'$COUNT'
    }
}

pipeline = [match, project, group, sort, limit, project_2]

list_documents = list(eu.aggregate(pipeline))

list_documents


[{'company': 'Uniwersyteckie Centrum Kliniczne', 'count': 14061},
 {'company': 'Szpital Kliniczny Przemienienia Pańskiego Uniwersytetu Medycznego im. Karola Marcinkowskiego w Poznaniu',
  'count': 12779},
 {'company': 'Viešoji įstaiga Centrinė projektų valdymo agentūra (126125624)',
  'count': 9956},
 {'company': 'Lesy Slovenskej republiky, štátny podnik', 'count': 9795},
 {'company': 'Szpital Uniwersytecki w Krakowie', 'count': 9624},
 {'company': 'Wojewódzki Szpital Zespolony', 'count': 9457},
 {'company': 'Klinički bolnički centar Rijeka', 'count': 9144},
 {'company': 'Spitalul Clinic Judetean de Urgenta Sf. Spiridon Iasi',
  'count': 8925},
 {'company': 'Wojskowy Instytut Medyczny', 'count': 8322},
 {'company': 'Kompania Węglowa S.A.', 'count': 8282},
 {'company': 'Institutul Oncologic Prof. Dr. I. Chiricuta Cluj-Napoca',
  'count': 8088},
 {'company': 'Sabiedrība ar ierobežotu atbildību “Ogres rajona slimnīca”',
  'count': 8049},
 {'company': 'Valsts reģionālās attīstības aģentūra

19. For each country get the highest company ('CAE_NAME') in terms of 'VALUE_EURO' sum contract spending

In [ ]:
match = {
    '$match': {
            '$and': [{'YEAR': {'$gte': bot_year}}, {'YEAR': {'$lte': top_year}}],
            'ISO_COUNTRY_CODE': {'$in': country_list}
        }
}

project = {
    '$project':{
        '_id':False,
        'CAE_NAME':True,
        'ISO_COUNTRY_CODE':True,
        'VALUE_EURO':True
    }
}

group_cpv_country_euro_avg = {
    '$group':{
        '_id':{'CAE_NAME':'$CAE_NAME','COUNTRY':'$ISO_COUNTRY_CODE'},
        'SUM':{'$sum':'$VALUE_EURO'}
    }
}

sort = {
    '$sort':{
        '_id.COUNTRY':1,
        'SUM':-1
    }
}

group_2 = {
    '$group':{
        '_id': '$_id.COUNTRY',
        'docs': {'$push': '$$ROOT'}
    }
}

project_2 = {
    '$project':{
        'top_1':{
            '$slice':['$docs', 1]
        }
    }
}

unwind = {
    '$unwind':'$top_1'
}

project_3 = {
    '$project':{
        '_id':False,
        'company':'$top_1._id.CAE_NAME',
        'sum':'$top_1.SUM',
        'country':'$_id'
    }
}

pipeline = [match, project, group_cpv_country_euro_avg, sort, group_2, project_2, unwind, project_3]

list_documents = list(eu.aggregate(pipeline))

list_documents

20. Returns the top 5 most frequent co-occurring companies ('CAE_NAME' and 'WIN_NAME')